In [1]:
import numpy as np
import pandas as pd


from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import  OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

In [3]:
print(len(train.columns), len(test.columns))

68 67


In [4]:
# 나이는 따로 진행
# 매핑 딕셔너리 생성
age_mapping = {
    '만18-34세': 0,
    '만35-37세': 1,
    '만38-39세': 2,
    '만40-42세': 3,
    '만43-44세': 4,
    '만45-50세': 5,
    '알 수 없음': -1
}

# '시술당시 나이' 컬럼에 매핑 적용
train['시술 당시 나이'] = train['시술 당시 나이'].replace(age_mapping)
test['시술 당시 나이'] = test['시술 당시 나이'].replace(age_mapping)

In [5]:
y_train = train['임신 성공 여부']
train = train.drop('임신 성공 여부', axis=1)
print(len(train.columns), len(test.columns))
print(train.shape, y_train.shape)

67 67
(256351, 67) (256351,)


In [6]:
# y_train이 pandas Series 형태일 때 (NumPy 배열이면 먼저 pandas로 변환 필요)
y_train_series = pd.Series(y_train)  # 혹은 pd.DataFrame(y_train)으로 사용

# 클래스의 분포 확인
class_distribution = y_train_series.value_counts()

# 비율로 확인
class_distribution_percentage = y_train_series.value_counts(normalize=True) * 100

# 결과 출력
print("클래스 분포:")
print(class_distribution)

print("\n클래스 분포(백분율):")
print(class_distribution_percentage)

클래스 분포:
임신 성공 여부
0    190123
1     66228
Name: count, dtype: int64

클래스 분포(백분율):
임신 성공 여부
0    74.16511
1    25.83489
Name: proportion, dtype: float64


In [7]:
# 카테고리 

categorical_columns = [
    "시술 시기 코드",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

In [8]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = train.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(train[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [9]:
X_train_encoded.shape

(256351, 67)

In [10]:
X_train_encoded[categorical_columns].head()

,시술 시기 코드,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,...,난자 출처,정자 출처,난자 기증자 나이,정자 기증자 나이,동결 배아 사용 여부,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,PGD 시술 여부,PGS 시술 여부
0,6.0,1.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,1.0,1.0
1,5.0,1.0,4.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,1.0,1.0
2,3.0,1.0,15.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,1.0,1.0
3,2.0,1.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,1.0,1.0
4,3.0,1.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,1.0,1.0


In [11]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [12]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [13]:
X_train_encoded.head()

,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,...,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,PGD 시술 여부,PGS 시술 여부,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일
0,6.0,0,0.0,1.0,4.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,0.0
1,5.0,5,0.0,1.0,4.0,0.0,3.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0,0.0,1.0,15.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0
3,2.0,1,0.0,1.0,4.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0,0.0,1.0,4.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,0.0


In [14]:
print(X_train_encoded.shape, y_train.shape)

(256351, 67) (256351,)


In [15]:
train.columns

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '특정 시술 유형',
       '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부',
       '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
       '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
       '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수',
       '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
       '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
       '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처',
       '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
       '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자

## 논문에서 사용한 features
- Patient Age
- Total number of previous cycles
- Total number of IVF pregnancies
- Total number of live births- conceived through IVF
- Type of infertility—female primary
- Type of Infertility—female secondary
- Type of infertility—male primary
- Type of infertility—male secondary
- Type of infertility—couple primary
- Type of infertility—couple secondary
- Cause of infertility—tubal disease
- Cause of infertility—ovulatory disorder
- Cause of infertility—male factor
- Cause of infertility—patient unexplained
- Cause of infertility—endometriosis
- Cause of infertility—cervical factors
- Cause of infertility—female factors
- Cause of infertility—partner sperm concentration
- Cause of infertility—partner sperm morphology
- Cause of infertility—partner sperm motility
- Cause of infertility—partner sperm immunological factors
- Stimulation used
- Egg source
- Sperm source
- Fresh cycle
- Frozen cycle
- Eggs thawed
- Fresh eggs collected
- Eggs mixed with partner sperm Embryos transferred

In [16]:
# 매핑된 컬럼 이름 리스트 (한국어)
selected_columns = [
    '시술 당시 나이', '총 시술 횟수', 'IVF 시술 횟수', 'IVF 출산 횟수',
    '여성 주 불임 원인', '여성 부 불임 원인', '남성 주 불임 원인', '남성 부 불임 원인',
    '부부 주 불임 원인', '부부 부 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애',
    '불임 원인 - 남성 요인', '불명확 불임 원인', '불임 원인 - 자궁내막증', '불임 원인 - 자궁경부 문제',
    '불임 원인 - 여성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 형태', '불임 원인 - 정자 운동성',
    '불임 원인 - 정자 면역학적 요인', '배란 유도 유형', '난자 출처', '정자 출처', '신선 배아 사용 여부',
    '동결 배아 사용 여부', '해동 난자 수', '수집된 신선 난자 수', '파트너 정자와 혼합된 난자 수', '이식된 배아 수'
]

# train, test 데이터프레임에서 선택된 컬럼만 필터링
filtered_train = X_train_encoded[selected_columns]
filtered_test = X_test_encoded[selected_columns]

# 결과 확인
filtered_train.head()

,시술 당시 나이,총 시술 횟수,IVF 시술 횟수,IVF 출산 횟수,여성 주 불임 원인,여성 부 불임 원인,남성 주 불임 원인,남성 부 불임 원인,부부 주 불임 원인,부부 부 불임 원인,...,불임 원인 - 정자 면역학적 요인,배란 유도 유형,난자 출처,정자 출처,신선 배아 사용 여부,동결 배아 사용 여부,해동 난자 수,수집된 신선 난자 수,파트너 정자와 혼합된 난자 수,이식된 배아 수
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,3.0,1.0,0.0,0.0,7.0,5.0,2.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,1.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,3.0,1.0,0.0,0.0,8.0,7.0,2.0
3,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,3.0,1.0,0.0,0.0,5.0,4.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,3.0,1.0,0.0,0.0,7.0,6.0,2.0


In [17]:
# # 데이터 스케일링
# scaler = StandardScaler()
# # X_scaled = scaler.fit_transform(X_train_encoded)
# X_scaled = scaler.fit_transform(filtered_train)
# X_test_scaled = scaler.transform(filtered_test)

# 데이터를 CNN 입력 형태로 변환 (2D 이미지 형태로 reshape)
# DataFrame을 NumPy 배열로 변환하여 reshape
X_scaled_reshaped = filtered_train.values.reshape(filtered_train.shape[0], filtered_train.shape[1], 1)
X_test_scaled_reshaped = filtered_test.values.reshape(filtered_test.shape[0], filtered_test.shape[1], 1)


# 데이터 나누기
X_train, X_val, y_train, y_val = train_test_split(X_scaled_reshaped, y_train, test_size=0.2, random_state=42)
X_train.shape

(205080, 30, 1)

In [18]:
print(X_scaled_reshaped.shape, y_train.shape)

(256351, 30, 1) (205080,)


In [19]:
model = Sequential()

# 첫 번째 Conv1D 계층 (1D 컨볼루션)
model.add(Conv1D(32, 1, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(BatchNormalization())  # BatchNormalization 추가
model.add(Dropout(0.2))  # Dropout 추가 (20% 비율로 드롭)
model.add(MaxPooling1D(2))

# 두 번째 Conv1D 계층
model.add(Conv1D(64, 1, activation='relu'))
model.add(BatchNormalization())  # BatchNormalization 추가
model.add(Dropout(0.2))  # Dropout 추가 (20% 비율로 드롭)
model.add(MaxPooling1D(2))

# 평탄화
model.add(Flatten())

# 완전 연결 계층 (Dense)
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())  # BatchNormalization 추가
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # 이진 분류 문제이므로 sigmoid 활성화 함수 사용

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()


# 모델 훈련
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Early Stopping 도입
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[lr_scheduler, early_stopping])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 30, 32)            64        
                                                                 
 batch_normalization (Batch  (None, 30, 32)            128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 30, 32)            0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 15, 32)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 15, 64)            2112      
                                                                 
 batch_normalization_1 (Bat  (None, 15, 64)            2

In [21]:
y_val_pred = model.predict(X_val)

# roc auc 점수 확인
roc_auc_score(y_val, y_val_pred)

1603/1603 [==============================] - 1s 676us/step


0.6368310246352009

In [22]:
y_pred = model.predict(X_test_scaled_reshaped)
y_pred

2815/2815 [==============================] - 2s 668us/step


array([[0.2662814 ],
       [0.26692525],
       [0.14530332],
       ...,
       [0.33391538],
       [0.18000409],
       [0.27231994]], dtype=float32)

In [ ]:
# 예측 결과는 (샘플 수, 1) 형식이므로, 1차원 배열로 변환합니다.
y_pred = y_pred.flatten()

# sample_submission.csv 파일 읽기
sample_submission = pd.read_csv('sample_submission.csv')

# 예측된 probability 값을 'probability' 열에 할당
sample_submission['probability'] = y_pred

# 결과를 새로운 CSV 파일로 저장
sample_submission.to_csv('submission.csv', index=False)

In [63]:
type(X_scaled)

numpy.ndarray